In [1]:
import pandas as pd
import os
import warnings
import time
warnings.filterwarnings("ignore")

import copy
from pathlib import Path
import warnings
import shutil

import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger
import numpy as np
import pandas as pd
import torch

import matplotlib.pyplot as plt

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet, RecurrentNetwork
from pytorch_forecasting.data import GroupNormalizer, NaNLabelEncoder
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss, MAPE, RMSE

import pickle
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters
from omegaconf import OmegaConf

torch.set_grad_enabled(True)

In [2]:
def get_csv_file_list(path):
    list_csv = os.listdir(path)
    return list_csv

def get_csv(path):
    df = pd.read_csv(path)
    #estado.drop(columns='Unnamed: 0',inplace=True)
    #estado.data = pd.to_datetime(estado.data)
    #estado = estado.loc[estado.data >= '2022-01-01']

    df['temperature'].fillna(method='ffill', inplace=True)
    df['windSpeed'].fillna(method='ffill', inplace=True)
    df['year'] = df['year'].astype(str)
    df['hour'] = df['hour'].astype(str)
    df['month'] = df['month'].astype(str)
    df['day'] = df['day'].astype(str)
    df['dayofweek_num'] = df['dayofweek_num'].astype(str)
    df['house_hold'] = df['house_hold'].astype(str)
    df['precipType'] = df['precipType'].astype(str)
    df['icon'] = df['icon'].astype(str)
    df['holiday'] = df['holiday'].astype(str)
    df['summary'] = df['summary'].astype(str)
    df['bool_weather_missing_values'] = df['bool_weather_missing_values'].astype(str)

    df = df.reset_index().rename(columns={'index':'time_idx'})
    return df 

def evaluation_metrics(val_dataloader,best_model,householde_name,execution_time):    
    predictions = best_model.predict(val_dataloader, return_y=True, trainer_kwargs=dict(accelerator="cpu"))
    mae = MAE()(predictions.output, predictions.y)
    mape = MAPE()(predictions.output, predictions.y)
    smape = SMAPE()(predictions.output, predictions.y)
    rmse = RMSE()(predictions.output, predictions.y)

    dict_ = {'House_Hold':householde_name.split(".")[0].upper(),
            'MAE':[mae.to('cpu').numpy().round(3)],
            'MAPE':[mape.to('cpu').numpy().round(3)],
            'SMAPE':[smape.to('cpu').numpy().round(3)],
            'RMSE':[rmse.to('cpu').numpy().round(3)],
            'Time_execution':f"{execution_time:.2f}s"}

    return dict_


def get_attention_values(interpretation,householde_name,encoder_list,decoder_list):
  att_encoder_values = interpretation['encoder_variables'].to('cpu').numpy()
  att_decoder_values = interpretation['decoder_variables'].to('cpu').numpy()
  tft_encoder = encoder_list
  tft_decoder = decoder_list
  encoder_dict = {}
  decoder_dict = {}
  for value, name in zip(att_encoder_values,tft_encoder):   
    encoder_dict[name] = [f"{np.round(value,4)*100:.2f}%"]
  encoder_dict['House_Hold'] = [householde_name.split(".")[0].upper()]
  for value, name in zip(att_decoder_values,tft_decoder):   
    decoder_dict[name] = [f"{np.round(value,4)*100:.2f}%"]
  decoder_dict['House_Hold'] = [householde_name.split(".")[0].upper()]
  return encoder_dict,decoder_dict


def cleaning_eval_metrics_results(path_origin, path_destiny,model_name):
    list_csv = os.listdir(path_origin)
    df_list = []
    for csv in list_csv:
        if os.path.isfile(os.path.join(path_origin,csv)):
            print(csv)
            df_list.append(pd.read_csv(path_origin + "/" + csv))
        else:
           print('Nao e csv')
    concat_df = pd.concat(df_list)
    concat_df.to_csv(path_destiny + "/" + f"{model_name}_metrics_results.csv",index=False)

In [3]:
def run_RNN_model(df,
                  csv_file_name,
                  cell_type,
                  path_pred,
                  path_metrics_val,
                  covariates,
                  learning_rate = .1,
                  hidden_size = 15,
                  dropout = .2,
                  loss = MAE(),
                  optimizer = "Ranger",
                  rnn_layers  = 2,
                  patience=10,
                  max_prediction_length = 168,
                  max_encoder_length = 720,
                  batch_size = 128,
                  seed = 81):

    df['Energy_kwh'] = df['Energy_kwh'].astype('float32')
    df['time_idx'] = df['time_idx'].astype('int32')
    training_cutoff = df["time_idx"].max() - max_prediction_length

    pl.seed_everything(seed)

    if covariates == ["time_idx"]:
        training = TimeSeriesDataSet(
                                df[lambda x: df.time_idx <= training_cutoff],
                                time_idx = 'time_idx',
                                target = 'Energy_kwh',
                                group_ids = ['house_hold'],
                                time_varying_known_reals=['time_idx'],
                                time_varying_unknown_reals = ['Energy_kwh'],
                                static_categoricals=['house_hold'],
                                min_encoder_length = max_encoder_length // 2,
                                max_encoder_length = max_encoder_length,
                                min_prediction_length=1,
                                max_prediction_length = max_prediction_length,
                                categorical_encoders = {'house_hold': NaNLabelEncoder(add_nan=True, warn=True)}
        )
    else:
        training = TimeSeriesDataSet(
                                    df[lambda x: df.time_idx <= training_cutoff],
                                    time_idx = 'time_idx',
                                    target = 'Energy_kwh',
                                    group_ids = ['house_hold'],
                                    #time_varying_known_reals=['time_idx'],
                                    time_varying_unknown_reals = ['Energy_kwh'],
                                    static_categoricals=['house_hold'],
                                    time_varying_known_categoricals = covariates,
                                    min_encoder_length = max_encoder_length // 2,
                                    max_encoder_length = max_encoder_length,
                                    min_prediction_length=1,
                                    max_prediction_length = max_prediction_length,
                                    categorical_encoders = {'house_hold': NaNLabelEncoder(add_nan=True, warn=True)}
        )


    validation = TimeSeriesDataSet.from_dataset(training, 
                                                df,
                                                predict = True,
                                                stop_randomization = True)


    train_dataloader = training.to_dataloader(train = True,
                                            batch_size = batch_size,
                                            num_workers = 1)


    val_dataloader = validation.to_dataloader(train = False,
                                            batch_size = batch_size,
                                            num_workers = 1)

    rnn = RecurrentNetwork.from_dataset(
                                            training,
                                            cell_type =cell_type,
                                            learning_rate = learning_rate,
                                            hidden_size = hidden_size,
                                            dropout = dropout,
                                            loss = loss,
                                            optimizer = optimizer,
                                            rnn_layers  = rnn_layers
        )

    early_stop_callback = EarlyStopping(monitor = "val_loss",
                                    min_delta = 0.0001,
                                    patience = patience,
                                    verbose = True,
                                    mode = "min")


    lr_logger = LearningRateMonitor()
    logger_LSTM = TensorBoardLogger(f"{cell_type}_logs")

    checkpoint_callback = ModelCheckpoint(monitor="val_loss")

    trainer = pl.Trainer(
                            max_epochs = 350,
                            accelerator = 'gpu',
                            enable_model_summary = True,
                            limit_train_batches = 300,
                            gradient_clip_val = 0.1,
                            callbacks = [lr_logger, early_stop_callback, checkpoint_callback],
                            logger = logger_LSTM,
                            enable_progress_bar=False
            )
    
    start_time = time.time()
    trainer.fit(
                    rnn,
                    train_dataloaders = train_dataloader,
                    val_dataloaders = val_dataloader)
    end_time = time.time()  
    execution_time = end_time - start_time  

    best_model_path = trainer.checkpoint_callback.best_model_path
    best_rnn = RecurrentNetwork.load_from_checkpoint(best_model_path)

    best_model_path = str(best_model_path)
    best_rnn = RecurrentNetwork.load_from_checkpoint(best_model_path)

    predictions = best_rnn.predict(val_dataloader, mode = "raw", return_x = True)

    df_predictions = pd.DataFrame({'time_idx':predictions.x['decoder_time_idx'][0].to('cpu').numpy(),
                                'Real':predictions.x['decoder_target'][0].to('cpu').numpy().round(3),
                                'predict':predictions.output[0][0].to('cpu').numpy().round(3).squeeze()})

    df_predictions.to_csv(path_pred + csv_file_name, index=False)

    eval_dict = evaluation_metrics(val_dataloader,best_rnn,csv_file_name,execution_time)
    df_eval_metrics = pd.DataFrame(eval_dict)
    df_eval_metrics.to_csv(path_metrics_val + csv_file_name, index=False)
    
    #shutil.rmtree('lightning_logs')
    #shutil.rmtree('LSTM_logs')

In [4]:
import os

def verificar_csvs_em_subpastas(pasta_principal, lista_csvs):
    """
    Verifica se cada arquivo CSV da lista existe em todas as subpastas de uma pasta principal.
    Retorna apenas os nomes dos arquivos que estão presentes em todas as subpastas.
    
    Args:
        pasta_principal (str): Caminho da pasta principal
        lista_csvs (list): Lista com os nomes dos arquivos CSV a serem procurados
        
    Returns:
        list: Lista com os nomes dos arquivos CSV que foram encontrados em todas as subpastas
    """
    # Lista para armazenar os arquivos que estão em todas as subpastas
    csvs_validos = []
    
    # Lista para armazenar todas as subpastas
    subpastas = []
    
    # Percorre a pasta principal para encontrar todas as subpastas
    for item in os.listdir(pasta_principal):
        caminho_completo = os.path.join(pasta_principal, item)
        if os.path.isdir(caminho_completo):
            subpastas.append(caminho_completo)
    
    # Se não houver subpastas, retorna lista vazia
    if not subpastas:
        return []
    
    # Para cada CSV da lista, verifica se ele existe em todas as subpastas
    for nome_csv in lista_csvs:
        # Adiciona extensão .csv se não estiver presente
        if not nome_csv.endswith('.csv'):
            nome_csv = f"{nome_csv}.csv"
            
        csv_presente_em_todas = True
        
        # Verifica se o arquivo existe em cada subpasta
        for subpasta in subpastas:
            caminho_arquivo = os.path.join(subpasta, nome_csv)
            if not os.path.isfile(caminho_arquivo):
                csv_presente_em_todas = False
                break
        
        # Se o arquivo estiver em todas as subpastas, adiciona à lista de válidos
        if csv_presente_em_todas:
            csvs_validos.append(nome_csv)
    
    return csvs_validos

def escrever_nome_csv(nome_csv, arquivo_txt='csv_.txt', modo='a'):
    """
    Escreve o nome de um arquivo CSV em um arquivo TXT.
    
    Args:
        nome_csv (str): Nome do arquivo CSV
        arquivo_txt (str): Nome do arquivo TXT de saída (default: 'nomes_csv.txt')
        modo (str): Modo de escrita - 'a' para append, 'w' para sobrescrever (default: 'a')
    """
    try:
        # Adiciona extensão .csv se não estiver presente
        if not nome_csv.endswith('.csv'):
            nome_csv = f"{nome_csv}.csv"
            
        # Abre o arquivo no modo especificado
        with open(arquivo_txt, modo, encoding='utf-8') as f:
            # Escreve o nome do CSV e adiciona uma quebra de linha
            f.write(f"{nome_csv}")
            
        print(f"Nome '{nome_csv}' escrito com sucesso em '{arquivo_txt}'")
            
    except Exception as e:
        print(f"Erro ao escrever no arquivo: {str(e)}")

def ler_nomes_csv(arquivo_txt='csv_.txt'):
    """
    Lê os nomes dos arquivos CSV de um arquivo TXT e retorna uma lista.
    
    Args:
        arquivo_txt (str): Nome do arquivo TXT para ler (default: 'nomes_csv.txt')
        
    Returns:
        list: Lista com os nomes dos arquivos CSV
    """
    lista_csvs = []
    
    try:
        # Abre o arquivo para leitura
        with open(arquivo_txt, 'r', encoding='utf-8') as f:
            # Lê cada linha do arquivo
            for linha in f:
                # Remove espaços em branco e quebras de linha
                nome_csv = linha.strip()
                
                # Verifica se a linha não está vazia
                if nome_csv:
                    # Adiciona extensão .csv se não estiver presente
                    if not nome_csv.endswith('.csv'):
                        nome_csv = nome_csv
                    
                    lista_csvs.append(nome_csv)
                    
        #print(f"Foram lidos {len(lista_csvs)} nomes de arquivos CSV")
        return lista_csvs
        
    except FileNotFoundError:
        print(f"Arquivo '{arquivo_txt}' não encontrado")
        return []
    except Exception as e:
        print(f"Erro ao ler o arquivo: {str(e)}")
        return []



In [5]:
def get_config(yaml_name):
    conf = OmegaConf.load(yaml_name)
    return conf

def get_covariate_combination(path):
    """ 
    look at eval_metric folder to get all combinations of covariates and return all combinations in a list
    """
    path = config.paths.eval_folder
    combinations = os.listdir(path)
    feature_combination = {}
    for combination in combinations:
        feature_combination[combination] = combination.split('-')
    return feature_combination


config = get_config("gru_config.yaml")

feature_combination = get_covariate_combination(config.paths.eval_folder)

In [6]:
feature_combination = {'#': ['#'],
                        'dayofweek_num': ['dayofweek_num'],
                        'hour': ['hour'],
                        'holiday': ['holiday'],
                        'precipType': ['precipType'],
                        'month': ['month'],
                        'bool_weather_missing_values': ['bool_weather_missing_values']
                    }

In [7]:
verificar_csvs_em_subpastas("eval_metrics",os.listdir("../EDA/dataframe_model/"))

['stretchedSociety_block_98_MAC004892.csv',
 'establishedAffluence_block_11_MAC001980.csv',
 'establishedAffluence_block_12_MAC002752.csv',
 'stretchedSociety_block_99_MAC004745.csv',
 'stretchedSociety_block_98_MAC004064.csv',
 'establishedAffluence_block_32_MAC001433.csv',
 'stretchedSociety_block_100_MAC002898.csv',
 'stretchedSociety_block_97_MAC001314.csv',
 'stretchedSociety_block_100_MAC003020.csv',
 'establishedAffluence_block_19_MAC005426.csv',
 'stretchedSociety_block_97_MAC000696.csv',
 'establishedAffluence_block_14_MAC002635.csv',
 'stretchedSociety_block_94_MAC000838.csv',
 'establishedAffluence_block_30_MAC003114.csv',
 'stretchedSociety_block_105_MAC002128.csv',
 'establishedAffluence_block_21_MAC004531.csv',
 'stretchedSociety_block_102_MAC003996.csv',
 'stretchedSociety_block_96_MAC001243.csv',
 'establishedAffluence_block_19_MAC002250.csv']

In [8]:
ler_nomes_csv()

['stretchedSociety_block_91_MAC000350.csv',
 'establishedAffluence_block_26_MAC002966.csv',
 'establishedAffluence_block_37_MAC000782.csv',
 'establishedAffluence_block_17_MAC002007.csv',
 'establishedAffluence_block_23_MAC004625.csv',
 'stretchedSociety_block_93_MAC004547.csv',
 'stretchedSociety_block_98_MAC001062.csv',
 'stretchedSociety_block_98_MAC001062.csv',
 'stretchedSociety_block_99_MAC004799.csv',
 'stretchedSociety_block_97_MAC001035.csv',
 'stretchedSociety_block_97_MAC001035.csv',
 'establishedAffluence_block_10_MAC004178.csv',
 'establishedAffluence_block_10_MAC001869.csv',
 'stretchedSociety_block_104_MAC000090.csv',
 'establishedAffluence_block_19_MAC002259.csv',
 'establishedAffluence_block_26_MAC003075.csv',
 'stretchedSociety_block_94_MAC000870.csv',
 'establishedAffluence_block_35_MAC001111.csv',
 'establishedAffluence_block_35_MAC001111.csv',
 'stretchedSociety_block_97_MAC001041.csv',
 'stretchedSociety_block_103_MAC004513.csv',
 'establishedAffluence_block_23_MA

In [9]:

for csv in os.listdir(config.paths.data_dir):
    csv_treinados = verificar_csvs_em_subpastas("eval_metrics",os.listdir("../EDA/dataframe_model/"))
    csv_problematicos = ler_nomes_csv()
    if csv in csv_problematicos:
        continue
    elif csv in csv_treinados:
        continue
    for folder_name,comb_features in feature_combination.items():
        if comb_features[0] == "#":
            comb_features = ['time_idx']
        if csv in csv_problematicos:
            continue
        elif csv in csv_treinados:
            continue
        try:
            df = get_csv(config.paths.data_dir + csv)
            print(f"Training model for {folder_name} using {csv}")
            print(len(df)-168)
            run_RNN_model(df,
                            max_prediction_length = 168,
                            max_encoder_length = len(df)-168,
                            covariates = comb_features,
                            csv_file_name = csv,
                            cell_type = "GRU",
                            path_pred = config.paths.pred_folder + folder_name+'/',
                            path_metrics_val=config.paths.eval_folder + folder_name+'/')
        except:
            escrever_nome_csv(csv)
            print(f"{csv} adicionado a lista de pdv's problematicos ")

Seed set to 81


Training model for # using stretchedSociety_block_91_MAC001165.csv
9985


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4050 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
2024-12-02 05:58:48.465010: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-02 05:58:48.824717: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following inst

Training model for dayofweek_num using stretchedSociety_block_91_MAC001165.csv
9985


Metric val_loss improved. New best score: 0.071
Monitored metric val_loss did not improve in the last 10 records. Best score: 0.071. Signaling Trainer to stop.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 81
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type           | Params | Mode 
------------------------------------------------------------
0 | loss             | MAE            | 0      | train
1 | logging_metrics  | ModuleList     | 0      | train
2 | embeddings       | MultiEmbedding | 218    | train
3 | rnn              | GRU            | 2.7 K  | train
4 | output_projector | Linear         | 16     | train


...

Seed set to 81
Seed set to 81


Nome 'establishedAffluence_block_40_MAC000459.csv' escrito com sucesso em 'csv_.txt'
establishedAffluence_block_40_MAC000459.csv adicionado a lista de pdv's problematicos 
Training model for holiday using establishedAffluence_block_40_MAC000459.csv
9985
Nome 'establishedAffluence_block_40_MAC000459.csv' escrito com sucesso em 'csv_.txt'
establishedAffluence_block_40_MAC000459.csv adicionado a lista de pdv's problematicos 
Training model for precipType using establishedAffluence_block_40_MAC000459.csv
9985


Seed set to 81
Seed set to 81


Nome 'establishedAffluence_block_40_MAC000459.csv' escrito com sucesso em 'csv_.txt'
establishedAffluence_block_40_MAC000459.csv adicionado a lista de pdv's problematicos 
Training model for month using establishedAffluence_block_40_MAC000459.csv
9985
Nome 'establishedAffluence_block_40_MAC000459.csv' escrito com sucesso em 'csv_.txt'
establishedAffluence_block_40_MAC000459.csv adicionado a lista de pdv's problematicos 
Training model for bool_weather_missing_values using establishedAffluence_block_40_MAC000459.csv
9985


Seed set to 81
